## References

- http://www.juliaopt.org/SumOfSquares.jl/latest/
- https://www.cs.colorado.edu/~xich8622/papers/thesis.pdf

## Support function of an interval

In this section, as a proof-of-principle, we compute the support function of an interval using SOS.

In [2]:
using SumOfSquares, DynamicPolynomials, MosekTools, TaylorModels, Plots

The support function of a set $X$ along direction $d$ is defined as the solution of the optimization problem:

$$
\rho(d, X) = \max \langle d, x \rangle \qquad s.t. x \in X.
$$
It represents how much the set $X$ is placed along direction $d$.

In [3]:
function sf(d::Vector, X::Interval)
    model = SOSModel(with_optimizer(Mosek.Optimizer, QUIET=true))
    @variable(model, x[1])
    @constraint(model, inf(X) <= x[1])
    @constraint(model, x[1] <= sup(X))
    @objective(model, Max, d[1] * x[1])
    optimize!(model)
    
    return objective_value(model)
end

sf (generic function with 1 method)

Let's consider an example:

In [4]:
X = Interval(-2.0, 7.0)

[-2, 7]

In [5]:
sf([1.0], X)

7.0

In [6]:
sf([-1.0], X)

2.0

## Support function of a TaylorModel1

Now we consider the more general case of a univariate taylor model.

The set defined by a TM is $S := \{ x : x = p(x_0) + y \wedge x_0 \in \textrm{dom(TM)} \wedge y \in \textrm{rem(TM)}\}$.

We would like to compute $\max \langle d, x\rangle$, such that $x \in S$, where $S$ is the range of the taylor model.

To fix ideas, consider the following example.

In [7]:
p = Taylor1([1.0, 1.0, 1.0], 6)

 1.0 + 1.0 t + 1.0 t² + 𝒪(t⁷)

In [8]:
rem = Interval(-0.1, 0.1)
x0 = Interval(0, 0)
dom = Interval(-2.0, 2.0)
X = TaylorModel1(p, rem, x0, dom)

 1.0 + 1.0 t + 1.0 t² + [-0.100001, 0.100001]

In [ ]:
plot(X, lab="")

For the support function along direction $d = [1.0]$, we expect $7.1$, obtained when $x_0 = 2$ and $y = 0.1$.

For the support function along direction $d = [-1.0]$, we expect $0.65$, obtained when $x_0 = -0.5$ and $y = -0.1$.

#### Using Ipopt

In [ ]:
using Ipopt

In [ ]:
d = [1.0]; # direction

In [ ]:
model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))

Xdom = domain(X)
Xrem = remainder(X)
@variable(model, inf(Xdom) <= x0 <= sup(Xdom))
@variable(model, inf(Xrem) <= y <= sup(Xrem))
@variable(model, x)
@NLconstraint(model, x == 1.0 + x0 + 1.0*x0^2 + y)
@objective(model, Max, d[1] * x)
model

In [ ]:
optimize!(model)
@show objective_value(model);

#### Using SumOfSquares

In [9]:
using SumOfSquares, DynamicPolynomials, MosekTools, SemialgebraicSets

To model the optimization problem using sum-of-squares, we introduce the variable $\gamma$ to represent the objective value bound, implemented as a JuMP decision variable, and introduce the constraint $\langle d, x\rangle \leq \gamma$. The objective function is set to be the minimum over all possible $\gamma$. The minimum upper bound under the polynomial restriction is the value we are looking for.

In [43]:
model = SOSModel(with_optimizer(Mosek.Optimizer, QUIET=true))
Xdom, Xrem = domain(X), remainder(X)
d = [1.0]

@polyvar x0 x y

S = @set x == 1.0 + x0 + 1.0*x0^2 + y &&
         inf(Xdom) <= x0 && x0 <= sup(Xdom) &&
         inf(Xrem) <= y && y <= sup(Xrem)

@variable(model, γ)
@constraint(model, d[1]*x + 0.0*y + 0.0*x0 <= γ, domain=S, maxdegree=3)

@objective(model, Min, γ)
optimize!(model)
@show objective_value(model);

objective_value(model) = 7.099999866012456


## [WIP] Polytopic overapproximation

Here we wrap this approach into a function and compute the 

In [ ]:
model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))

Xdom = domain(X)
Xrem = remainder(X)
@variable(model, inf(Xdom) <= x0 <= sup(Xdom))
@variable(model, inf(Xrem) <= y <= sup(Xrem))
@variable(model, x)
@NLconstraint(model, x == sum(x0^i * p.coeffs[i] for i in 1:1+get_order(p)) + y)
@objective(model, Max, d[1] * x)
model

In [ ]:
@NLconstraint(model, sum(x0^i * p.coeffs[i] for i in 1:1+get_order(p)) == 0)

In [ ]:
S = sum(x0^i * p.coeffs[i] for i in 1:1+get_order(p))